In [1]:
import sys
sys.path.append("..")
from allthethings import PyNetwork, PyPipe_ps
from allthethings import PyBC_opt_dh
import numpy as np
import matplotlib.pyplot as plt
%pylab inline
from scipy import optimize
from writeit import rewritePipes

Populating the interactive namespace from numpy and matplotlib


In [2]:
def idx_t(i,j,n,N):
    return (2*(N+2)*n+(N+2)*i+j)

In [3]:
fi = "../indata/Alameda62.0.0.inp" 
fc = "../indata/Alameda62.0.config"
n0 = PyNetwork(fi,fc,1)
print n0.Ls
print n0.Ns
print min(n0.Ds)

[ 116.43  214.15   57.79   62.33   70.59   37.98   73.46   77.42   75.9
  101.19   77.72   82.91   41.45   39.93   75.22  108.51  104.24   58.83
  103.33  101.5   123.14   40.84   58.83   19.81   20.42  256.03  130.76
   10.36   60.96   91.44   13.11   60.96   91.44]
[24 43 12 13 15  8 15 16 16 21 16 17  9  8 16 22 21 12 21 21 25  9 12  4  5
 52 27  3 13 19  3 13 19]
0.051


In [4]:
Ls = n0.Ls
Ds = n0.Ds
T = 20
Mi = 10   #number of time steps in between writes
Nt = 20
Ttot = Nt*T
Np = n0.Nedges
a = 100
Ns = [int(l) for l in Ls]
dx = [Ls[i]/Ns[i] for i in range(Np)]
M = int(T*a/(max(dx)*.8))*10
M = (M+Mi-M%Mi)  #round it up to be an even multiple of Mi
jt = n0.nodeTypes
Nn = len(jt)
bt = [1]*Nn
bv = [0]*Nn
r = [1]*Nn
r[0] = 0
bt[0] =1
h0s = [0]*Np
h0s = [.1*d*d/4*np.pi for d in Ds]
q0s = [0]*Np
Mrs =[0.007]*Np
#Mrs =[0.0]*Np
print T
print M
Nstar =1 #measuring point for each pipe
Hs =np.ndarray((Np,M/Mi*Nt))
print (T/float(M))/min(dx)*a
print min(Ns)
elevs = [127.10, 126.19, 98.76, 91.44, 83.82, 78.029, 70.74, 61.26, 53.95, 57.9, 58.52, 60.05, 60.66, 62.79, 63.70, 73.15, 76.81, 76.2, 86.26, 84.43, 88.09, 96.32, 96.012, 99.67, 101.80, 103.63, 101.80, 101.19, 101.50, 103.95, 117.96, 129.84, 87.48, 97.23]

20
23980
0.0833930630718
10


In [5]:
fn = "../indata/Alameda_m3"
oldinp = "../indata/Alameda62.0.0.inp"
(fi, fc) = rewritePipes(fn,oldinp, Ns, Ls, Mrs, Ds, jt, bt, bv, r, h0s, q0s, T, M, a, elevs)
n1 = PyNetwork(fi,fc,1)
dt = n1.T/float(n1.M)
Q00 = 0.0087
#Q00 = 0.007
A00 = 0.00793
for i in range(0,Np):
    Ai = .0*(Ds[i]**2)/4*np.pi
    A0 = Ai*ones(n1.Ns[i])
    Q0 = 0*np.ones(n1.Ns[i])
    n1.setIC(i,A0,Q0)
n1.setIC(0,A00*np.ones(n1.Ns[0]), Q00*np.ones(n1.Ns[0]))
n1.setIC(1,A00*np.ones(n1.Ns[1]), Q00*np.ones(n1.Ns[1]))
Qb = Q00*np.ones(M+1)
Qo = 0.04*np.ones(M+1)
Ab = A00*np.ones(M+1)
#aa=arange(0,len(n1.nodeTypes))
#for j in range(len(aa[n1.nodeTypes==1])):
#    n1.setbVal(j,n1.Ds[i]*np.ones(M+1))
n1.setbVal(0,Qb)
print n1.Ds
print Ds
for i in range(n1.Nedges):
    print "%d    %d" %(i,n1.Ns[i])
p0 = PyPipe_ps(n1.Ns[11],n1.Ds[11], n1.Ls[11],M, a)

new files are ../indata/Alameda_m3.inp and ../indata/Alameda_m3.config
[ 0.1   0.1   0.1   0.1   0.1   0.1   0.1   0.1   0.1   0.1   0.1   0.1
  0.1   0.1   0.1   0.1   0.05  0.1   0.05  0.05  0.1   0.05  0.05  0.05
  0.05  0.05  0.05  0.1   0.1   0.05  0.1   0.1   0.05]
[ 0.102  0.102  0.102  0.102  0.102  0.102  0.102  0.102  0.102  0.102
  0.102  0.102  0.102  0.102  0.102  0.102  0.051  0.102  0.051  0.051
  0.102  0.051  0.051  0.051  0.051  0.051  0.051  0.102  0.102  0.051
  0.102  0.102  0.051]
0    116
1    214
2    57
3    62
4    70
5    37
6    73
7    77
8    75
9    101
10    77
11    82
12    41
13    39
14    75
15    108
16    104
17    58
18    103
19    101
20    123
21    40
22    58
23    19
24    20
25    256
26    130
27    10
28    60
29    91
30    13
31    60
32    91


In [8]:
#c= p0.Cgrav(1e-6,False) 
#print c
print n1.Ls[16]/n1.Ns[16]
print n1.Ds[16]
print dt
print np.pi*(.05**2)/4.

1.0
0.05
0.000834028356964
0.00196349540849


In [ ]:
%%time 
Vs = [n1.getTotalVolume()]
for m in range(Nt):
    n1.runForwardProblem(dt)
    for j in range(Np):
        N = n1.Ns[j]
        p0 = PyPipe_ps(N,n1.Ds[j], n1.Ls[j],M,a)
        qh = n1.qhist(j)
        Htemp = [p0.pbar(qh[idx_t(0,Nstar,n,n1.Ns[j])],False) for n in range(1,M+1,Mi)]
        Hs[j,m*(M/Mi):(M/Mi)*(m+1)] = Htemp
    Vs.append(n1.getTotalVolume()) 
    print m
    print Vs
    n1.reset()

0
[2.616899999999998, 2.7908999456773116]
1
[2.616899999999998, 2.7908999456773116, 3.115811492077269]
2
[2.616899999999998, 2.7908999456773116, 3.115811492077269, 3.6203563537091656]
3
[2.616899999999998, 2.7908999456773116, 3.115811492077269, 3.6203563537091656, 3.80336055561007]
4
[2.616899999999998, 2.7908999456773116, 3.115811492077269, 3.6203563537091656, 3.80336055561007, 4.097177481794783]
5

In [ ]:
m2psi = 1.42
t = linspace(0,Ttot,shape(Hs)[1])
for k in range(0,Np):
    plot(t,m2psi*Hs[k,:])
xlabel('t (s)')
ylabel('pressure head (psi)')

In [ ]:
interesting = [32]
for i in interesting:
    plot(t,m2psi*Hs[i,:],label= 'pipe %d, L = %.f m'%(i,n1.Ls[i]))
print dt
print a
#xticks(linspace(280,290,11))
grid(True)
xlabel('t')
ylabel('pressure head (psi)')
print n1.Ls[18]
print n1.Ls[19]
print n1.Ls[20]
legend(loc = 'upper left')
print n1.Ls

In [ ]:
# combination that works to 140 s: Q0 = 0.05Af, r = -1 everywhere.